In [1]:
%run "../../PyUtils/common.ipynb"
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcess
from sklearn import linear_model
from sklearn import cross_validation
from sklearn import ensemble
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
#from statsmodels.regression.quantile_regression import QuantReg
#import statsmodels.formula.api as smf
from sklearn.model_selection import learning_curve
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import r2_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

han1="http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20patient_data_table";
hand="http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20hand"
kimj="http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20kimj";
leeh="http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20leeh";
sony="http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20sony";
kuky="http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20kuky";

fileName = hand;

dfOriginal = LoadDataSet(fileName, checkForDateTime=False);
displayDFs (dfOriginal, maxrows=3 )
#for c in dfOriginal.columns: print (c,  end=', ')

<IPython.core.display.Javascript object>

In [2]:
df=dfOriginal.copy()

#1. Remove all the rows that does not have any pef values 
df.insert(4, 'npt',0)
df.insert(4, 'pef',0)
df.npt = pd.notnull(df.pef1)*1 + pd.notnull(df.pef2)*1 + pd.notnull(df.pef3)*1
df = df[df.npt > 0]
df=df.reset_index(drop=True)
df.pef = (df.pef1 + df.pef2 + df.pef3)/df.npt

#2. Combine 
df=df.fillna(0)
df.loc[df.timeofday == 'null', 'timeofday'] = "00:00:00"
sdttm = df.dateofmeasure + " " + df.timeofday 
df.insert(0, 'sdttm', sdttm)
df.sdttm = pd.to_datetime(df.sdttm)

df.sort_values(by='sdttm', ascending=True, inplace=True)
drps  = "cname, ampm, timeofday, dateofmeasure, npt, pef1, pef2, pef3, pef, indexpef".split(', ')
df=df.drop(drps, axis=1, errors='ignore')
df=df.reset_index(drop=True)

## <== do the following for SAP PA tool
#pef = df.pef;
#df=df.drop(['pef'], axis=1)
#df.insert(1, 'pef1', pef)

#pef[-10:]=0     # Set last 10 to zero for predictions
#df.insert(1, 'pef', pef)
#df.to_csv("HanD/hand1.csv", sep=';')

df1 = df.copy()
df1 = df1.set_index(df1.sdttm)
df1=df1.drop('sdttm', axis=1, errors='ignore')

s= pd.qcut(df1.pefmax, 10, labels='a1,a2,a3,a4,a5,a6,a7,a8,a9,a10'.split(','))
df1.insert(1,"pefcat", s)
df1.pefcat = df1.pefcat.astype(str)

columns = '''pefmax,so2,co,o3,no2,temperaturec,windspeedms,precipitationpercent,vaporpressurehpa,
dewpointtemperaturec,airpressurehpa,sealevelpressurehpa,groundtemperaturec,tmax,amax,tmin'''
#amin,pmin,tmaxlesstmin,amaxlessamin,pmaxlesspmin'''
cs = [c.strip() for c in columns.split(',')]

df1=df1[cs]
#HTML(df1.to_html())
displayDFs (df1, maxrows=3 )


In [17]:
df2 = df.copy()
df2=df2.rename(columns={"sdttm": "Date"})

columns = '''Date,pefmax,so2,co,o3,no2,temperaturec,windspeedms,precipitationpercent,vaporpressurehpa,
dewpointtemperaturec,airpressurehpa,sealevelpressurehpa,groundtemperaturec,tmax,amax,tmin'''
#amin,pmin,tmaxlesstmin,amaxlessamin,pmaxlesspmin'''
cs = [c.strip() for c in columns.split(',')]

df2.Date=df2.Date.astype(int)

#df3=df2['pefmax,so2,co,o3,no2,temperaturec,windspeedms,precipitationpercent,vaporpressurehpa'.split(',')]
df2.to_csv('hand.csv', index=False)

In [12]:
df2.Date.astype(int)

0       1419362940000000000
1       1419406800000000000
2       1419444360000000000
3       1419489660000000000
               ...         
1209    1475695260000000000
1210    1475733600000000000
1211    1475777460000000000
1212    1475824680000000000
Name: Date, Length: 1213, dtype: int64

In [15]:
pd.to_datetime(1419362940000000000)

Timestamp('2014-12-23 19:29:00')

In [16]:
df2

,Date,pefmax,so2,co,o3,no2,temperaturec,windspeedms,precipitationpercent,vaporpressurehpa,dewpointtemperaturec,airpressurehpa,sealevelpressurehpa,groundtemperaturec,tmax,amax,tmin
0,2014-12-23 19:29:00,339.000,0.188,23.400,0.063,1.272,2.000,1.300,65.000,4.600,-3.900,1020.000,1024.300,-0.100,3.700,1023.900,-6.900
1,2014-12-24 07:40:00,319.000,0.196,19.900,0.077,1.176,1.900,0.300,83.000,5.800,-0.600,1017.300,1021.600,-0.400,3.700,1023.900,-6.600
2,2014-12-24 18:06:00,316.000,0.214,19.800,0.151,1.042,2.100,2.200,56.000,4.000,-5.700,1018.200,1022.500,-0.100,7.900,1020.300,0.500
3,2014-12-25 06:41:00,326.000,0.180,18.400,0.378,0.750,-3.600,1.200,66.000,3.100,-9.000,1023.900,1028.300,-3.900,7.900,1023.800,-3.700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,2016-10-05 19:21:00,214.000,0.063,10.300,0.460,0.490,20.900,1.400,86.000,21.200,18.400,1008.200,1012.200,19.900,26.500,1010.300,17.200
1210,2016-10-06 06:00:00,229.000,0.067,10.500,0.632,0.407,14.400,1.700,83.000,13.600,11.500,1015.900,1020.000,15.000,26.500,1015.100,15.000
1211,2016-10-06 18:11:00,230.000,0.072,10.900,0.652,0.388,18.800,2.100,60.000,12.900,10.800,1017.800,1021.800,18.000,23.300,1017.800,13.800
1212,2016-10-07 07:18:00,271.000,0.072,11.400,0.390,0.548,17.300,2.100,83.000,16.300,14.300,1018.500,1022.500,16.300,23.300,1019.200,13.800


In [ ]:
import pyaf.ForecastEngine as autof
lEngine = autof.cForecastEngine()

lExogenousData = (df2 , cs[2:6]) 
df3=df2["Date,pefmax".split(',')].copy()


In [ ]:
lEngine.train(df3, 'Date' , 'pefmax', 12 , lExogenousData);

In [ ]:
import pyaf.ForecastEngine as autof
lEngine_Without_Exogenous = autof.cForecastEngine()
lEngine_Without_Exogenous.train(df3 , 'Date' , 'pefmax', 7);

In [ ]:
ozone_forecast_without_exog = lEngine_Without_Exogenous.forecast(ozone_dataframe, 12);
ozone_forecast_with_exog = lEngine.forecast(ozone_dataframe, 12);

In [ ]:
%matplotlib inline  
ozone_forecast_without_exog.plot.line('Date', ['Ozone' , 'Ozone_Forecast', 
                                             'Ozone_Forecast_Lower_Bound', 
                                             'Ozone_Forecast_Upper_Bound'], grid = True, figsize=(12, 8))
ozone_forecast_with_exog.plot.line('Date', ['Ozone' , 'Ozone_Forecast', 
                                             'Ozone_Forecast_Lower_Bound', 
                                             'Ozone_Forecast_Upper_Bound'], grid = True, figsize=(12, 8))